## 0 Set-ups

### 0.0 Install (only if needed)

In [ ]:
!aws codeartifact login --tool pip --repository aip-python --domain aip-python --domain-owner 569456961653 --region us-east-1
%pip install ammpy
%pip config unset global.index-url

### 0.0 Export results (run only if needed)

In [ ]:
# export results folder a ZIP file (this allows to download the whole folder at once)
cwd = os.getcwd()
os.chdir(cwd + '/results')
!tar chvfz results.tar.gz *
os.chdir(cwd)

### 0.1 Import modules and functions

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from ipywidgets import widgets, Layout
import pickle
import ammpy
from ammpy.s3 import File
from ammpy.aap.presentation import Presentation
from ammpy.locale import metadata

### 0.2 Import basic data

In [ ]:
# build directory if doesn't exist
path = './temporary_files'
if not os.path.exists(path):
    os.mkdir(path)

In [ ]:
# this imports latest repository of brands onboarded into automation
s3_file = File('s3://amm-sagemaker-data/ariju/bpr/bpr_brands.csv')
s3_file.download("./temporary_files") # Download the file to a local path, with custom name via second argument

In [ ]:
# pandas global options
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.6f}'.format

## 1 User input

In [ ]:
# create empty dic
input_dic = {}

In [ ]:
# import data on brands onboarded
df_bpr_inputs = pd.read_csv('./temporary_files/bpr_brands.csv')

### 1.2 User input

In [ ]:
# locale
widget_local = widgets.Dropdown(options=sorted(df_bpr_inputs.locale.unique()), description='Locale:', disabled=False); 
display(widget_local)

In [ ]:
widget_brand_name = widgets.Dropdown(options=sorted(df_bpr_inputs[df_bpr_inputs['locale']==widget_local.value].brand_name.unique()), description='Brand Name:', disabled=False); 
display(widget_brand_name)

In [ ]:
mask_1 = df_bpr_inputs['locale']==widget_local.value
mask_2 = df_bpr_inputs['brand_name']==widget_brand_name.value
widget_browse_node = widgets.Dropdown(options=sorted(df_bpr_inputs[mask_1 & mask_2].node_path.unique()), description='Brand Name:', disabled=False); 
display(widget_browse_node)

### 1.3 Fill input

In [ ]:
# filter input file
mask_1 = df_bpr_inputs['locale']==widget_local.value
mask_2 = df_bpr_inputs['brand_name']==widget_brand_name.value
mask_3 = df_bpr_inputs['node_path']==widget_browse_node.value
# populate dic
input_dic['local']=widget_local.value
input_dic['brand_name']=widget_brand_name.value
input_dic['node_path']=widget_browse_node.value
input_dic['marketplace_id']=int(df_bpr_inputs[mask_1 & mask_2 & mask_3].marketplace_id.item())
input_dic['root_browse_node_id']=int(df_bpr_inputs[mask_1 & mask_2 & mask_3].root_browse_node_id.item())
input_dic['browse_node_id']=int(df_bpr_inputs[mask_1 & mask_2 & mask_3].browse_node_id.item())
input_dic['category_friendly_name']=df_bpr_inputs[mask_1 & mask_2 & mask_3].category_friendly_name.item()
input_dic['category_url']= metadata().loc[widget_local.value,'website'] + '/b?node=' + str(input_dic['browse_node_id'])
input_dic['root_browse_node_name'] = input_dic['node_path'].split('/')[0]

In [ ]:
input_dic

### 1.4 Store inputs

In [ ]:
# build directories
path = [
    "./results",
    "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'],
    "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'],
    "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/input'
]
for p in path:
    if not os.path.exists(p):
        os.mkdir(p)

In [ ]:
# save dictionary on local environment
import pickle
# create a binary pickle file
path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/input'
file = 'user_input.pkl'
f = open(path + '/' + file,"wb")
# write the python object (dict) to pickle file
pickle.dump(input_dic,f)
# close file
f.close()

## 2 Import datasets

In [ ]:
# import (if exists) a file stored with all user input
import pickle
path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/input'
file = 'user_input.pkl'
if os.path.exists(path + '/' + file):
    file_to_read = open(path + '/' + file, "rb")
    input_dic = pickle.load(file_to_read)

### 2.1 Monthly data

In [ ]:
# import from
import_path = 's3://amm-sagemaker-data/ariju/bpr/' + str(input_dic['marketplace_id'])
import_file = 'monthly_data000'

# import from S3
s3_file = File(import_path + '/' + import_file)
s3_file.download("./temporary_files") # Download the file to a local path, with custom name via second argument

In [ ]:
# import from
df = pd.read_csv('./temporary_files/monthly_data000')

cond = df['report_brand'] == input_dic['brand_name']
cond_2 = df['report_root_browse_node_id'] == input_dic['root_browse_node_id']
cond_3 = df['report_browse_node_id'] == input_dic['browse_node_id']
df = df[cond & cond_2 & cond_3]

# export to
export_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/input'
export_file = 'data_monthly.csv'
df.to_csv(export_path + '/' + export_file, index=False)
del df

### 2.2 Weekly data

In [ ]:
# import from
import_path = 's3://amm-sagemaker-data/ariju/bpr/' + str(input_dic['marketplace_id'])
import_file = 'weekly_data000'

# import from S3
s3_file = File(import_path + '/' + import_file)
s3_file.download("./temporary_files") # Download the file to a local path, with custom name via second argument

In [ ]:
# import from
df = pd.read_csv('./temporary_files/weekly_data000')

cond = df['report_brand'] == input_dic['brand_name']
cond_2 = df['report_root_browse_node_id'] == input_dic['root_browse_node_id']
cond_3 = df['report_browse_node_id'] == input_dic['browse_node_id']
df = df[cond & cond_2 & cond_3]

# export to
export_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/input'
export_file = 'data_weekly.csv'
df.to_csv(export_path + '/' + export_file, index=False)
del df

## 3 Data manipulation

In [ ]:
# Create an empty dic to store results
output = {}

### 3.1 Import data 

In [ ]:
# import (if exists) a file stored with all user input
import pickle
path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/input'
file = 'user_input.pkl'
if os.path.exists(path + '/' + file):
    file_to_read = open(path + '/' + file, "rb")
    input_dic = pickle.load(file_to_read)

In [ ]:
# build directories
path = [
    "./results",
    "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'],
    "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'],
    "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
]
for p in path:
    if not os.path.exists(p):
        os.mkdir(p)

In [ ]:
# import file: monthly
import_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/input'
import_file = 'data_monthly.csv'
df_monthly = pd.read_csv(import_path + '/' + import_file)
df_monthly['year'] = df_monthly['year'].astype(int)
df_monthly['month_starting_date'] = pd.to_datetime(df_monthly['month_starting_date'])
df_monthly.dtypes

In [ ]:
# import file: monthly
import_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/input'
import_file = 'data_weekly.csv'
df_weekly = pd.read_csv(import_path + '/' + import_file)
df_weekly['week_starting_date'] = pd.to_datetime(df_weekly['week_starting_date'])
df_weekly.dtypes

### 3.2 Manipulated Tables

Period: T12M vs P12M  
Quarterly  
Monthly  
Weekly

#### Period: T12M vs P12M

In [ ]:
df = df_monthly\
    .groupby('period')\
    .agg({
        # brand
        'brand_ordered_revenue':'sum',
        'brand_ordered_quantity':'sum',
        'brand_sns_ordered_revenue':'sum',
        'brand_sns_quantity':'sum',
        'brand_glance_views':'sum',
        'brand_total_search_impressions':'sum',
        'brand_total_display_impressions':'sum',
        'brand_total_impressions':'sum',
        # peer set
        'peer_set_ordered_revenue':'sum',
        'peer_set_ordered_quantity':'sum',
        'peer_set_sns_ordered_revenue':'sum',
        'peer_set_sns_quantity':'sum',
        'peer_set_glance_views':'sum',
        'peer_set_total_search_impressions':'sum',
        'peer_set_total_display_impressions':'sum',
        'peer_set_total_impressions':'sum'
    })\
    .sort_index()

df

In [ ]:
# calculate deltas (%)
df['brand_revenue_delta'] = df['brand_ordered_revenue'] / df['brand_ordered_revenue'].shift() - 1
df['brand_quantity_delta'] = df['brand_ordered_quantity'] / df['brand_ordered_quantity'].shift() - 1
df['brand_sns_revenue_delta'] = df['brand_sns_ordered_revenue'] / df['brand_sns_ordered_revenue'].shift() - 1
df['brand_sns_quantity_delta'] = df['brand_sns_quantity'] / df['brand_sns_quantity'].shift() - 1
df['brand_gv_delta'] = df['brand_glance_views'] / df['brand_glance_views'].shift() - 1
df['brand_search_imp_delta'] = df['brand_total_search_impressions'] / df['brand_total_search_impressions'].shift() - 1
df['brand_display_imp_delta'] = df['brand_total_display_impressions'] / df['brand_total_display_impressions'].shift() - 1
df['brand_total_imp_delta'] = df['brand_total_impressions'] / df['brand_total_impressions'].shift() - 1

df['peer_set_revenue_delta'] = df['peer_set_ordered_revenue'] / df['peer_set_ordered_revenue'].shift() - 1
df['peer_set_quantity_delta'] = df['peer_set_ordered_quantity'] / df['peer_set_ordered_quantity'].shift() - 1
df['peer_set_sns_revenue_delta'] = df['peer_set_sns_ordered_revenue'] / df['peer_set_sns_ordered_revenue'].shift() - 1
df['peer_set_sns_quantity_delta'] = df['peer_set_sns_quantity'] / df['peer_set_sns_quantity'].shift() - 1
df['peer_set_gv_delta'] = df['peer_set_glance_views'] / df['peer_set_glance_views'].shift() - 1
df['peer_set_search_imp_delta'] = df['peer_set_total_search_impressions'] / df['peer_set_total_search_impressions'].shift() - 1
df['peer_set_display_imp_delta'] = df['peer_set_total_display_impressions'] / df['peer_set_total_display_impressions'].shift() - 1
df['peer_set_total_imp_delta'] = df['peer_set_total_impressions'] / df['peer_set_total_impressions'].shift() - 1

df

In [ ]:
# calculate shares
df['brand_revenue_share'] = (df.brand_ordered_revenue / df.peer_set_ordered_revenue).round(3)
df['brand_quantity_share'] = (df.brand_ordered_quantity / df.peer_set_ordered_quantity).round(3)
df['brand_sns_revenue_share'] = (df.brand_sns_ordered_revenue / df.peer_set_sns_ordered_revenue).round(3)
df['brand_sns_quantity_share'] = (df.brand_sns_quantity / df.peer_set_sns_quantity).round(3)
df['brand_gv_share'] = (df.brand_glance_views / df.peer_set_glance_views).round(3)
df['brand_search_sov'] = (df.brand_total_search_impressions / df.peer_set_total_search_impressions).round(3)
df['brand_display_sov'] = (df.brand_total_display_impressions / df.peer_set_total_display_impressions).round(3)
df['brand_total_sov'] = (df.brand_total_impressions / df.peer_set_total_impressions).round(3)

# calculate CVR
df['brand_cvr'] = (df.brand_ordered_quantity - df.brand_sns_quantity) / df.brand_glance_views
df['peer_set_cvr'] = (df.peer_set_ordered_quantity - df.peer_set_sns_quantity) / df.peer_set_glance_views

df

In [ ]:
# calculate share deltas (%)
df['brand_revenue_share_delta'] = df['brand_revenue_share'] / df['brand_revenue_share'].shift() - 1
df['brand_quantity_share_delta'] = df['brand_quantity_share'] / df['brand_quantity_share'].shift() - 1
df['brand_sns_revenue_share_delta'] = df['brand_sns_revenue_share'] / df['brand_sns_revenue_share'].shift() - 1
df['brand_sns_quantity_share_delta'] = df['brand_sns_quantity_share'] / df['brand_sns_quantity_share'].shift() - 1
df['brand_gv_share_delta'] = df['brand_gv_share'] / df['brand_gv_share'].shift() - 1
df['brand_search_sov_delta'] = df['brand_search_sov'] / df['brand_search_sov'].shift() - 1
df['brand_display_sov_delta'] = df['brand_display_sov'] / df['brand_display_sov'].shift() - 1
df['brand_total_sov_delta'] = df['brand_total_sov'] / df['brand_total_sov'].shift() - 1

df

In [ ]:
# calculate indexed deltas
df['brand_revenue_share_delta_index'] = df['brand_revenue_share'] / df['brand_revenue_share'].shift() * 100
df['brand_quantity_share_delta_index'] = df['brand_quantity_share'] / df['brand_quantity_share'].shift() * 100
df['brand_sns_revenue_share_delta_index'] = df['brand_sns_revenue_share'] / df['brand_sns_revenue_share'].shift() * 100
df['brand_sns_quantity_share_delta_index'] = df['brand_sns_quantity_share'] / df['brand_sns_quantity_share'].shift() * 100
df['brand_gv_share_delta_index'] = df['brand_gv_share'] / df['brand_gv_share'].shift() * 100
df['brand_search_sov_delta_index'] = df['brand_search_sov'] / df['brand_search_sov'].shift() * 100
df['brand_display_sov_delta_index'] = df['brand_display_sov'] / df['brand_display_sov'].shift() * 100
df['brand_total_sov_delta_index'] = df['brand_total_sov'] / df['brand_total_sov'].shift() * 100
# indexed cvr
df['brand_cvr_index'] = df['brand_cvr'] / df['peer_set_cvr'] * 100

# SnS contribution
df['brand_sns_revenue_contribution'] = df['brand_sns_ordered_revenue'] / df['brand_ordered_revenue'] * 100
df['peer_set_sns_revenue_contribution'] = df['peer_set_sns_ordered_revenue'] / df['peer_set_ordered_revenue'] * 100

df

In [ ]:
# reset index
df = df.reset_index()
df

In [ ]:
# export results
export_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
export_file = 'output_period.csv'
df.to_csv(export_path + '/' + export_file, index=False)
del df

#### Quarterly

In [ ]:
# if last quarter is not over
current_quarter = df_monthly['month_starting_date'].max().quarter
current_quarter = 'Q' + str(current_quarter)
current_year = df_monthly['month_starting_date'].max().year

if not int(df_monthly['month_starting_date'].max().strftime("%m")) in [3,6,9,12]:
    current_quarter_updated = current_quarter + ' QTD'
else:
    current_quarter_updated = current_quarter

In [ ]:
df = df_monthly\
    .groupby(['year','quarter'])\
    .agg({
        # brand
        'brand_ordered_revenue':'sum',
        'brand_ordered_quantity':'sum',
        'brand_sns_ordered_revenue':'sum',
        'brand_sns_quantity':'sum',
        'brand_glance_views':'sum',
        'brand_total_search_impressions':'sum',
        'brand_total_display_impressions':'sum',
        'brand_total_impressions':'sum',
        # peer set
        'peer_set_ordered_revenue':'sum',
        'peer_set_ordered_quantity':'sum',
        'peer_set_sns_ordered_revenue':'sum',
        'peer_set_sns_quantity':'sum',
        'peer_set_glance_views':'sum',
        'peer_set_total_search_impressions':'sum',
        'peer_set_total_display_impressions':'sum',
        'peer_set_total_impressions':'sum'
    })\
    .sort_index()

df.tail()

In [ ]:
# calculate deltas (%)
# df['brand_revenue_delta'] = df['brand_ordered_revenue'] / df['brand_ordered_revenue'].shift() - 1
# df['brand_quantity_delta'] = df['brand_ordered_quantity'] / df['brand_ordered_quantity'].shift() - 1
# df['brand_sns_revenue_delta'] = df['brand_sns_ordered_revenue'] / df['brand_sns_ordered_revenue'].shift() - 1
# df['brand_sns_quantity_delta'] = df['brand_sns_quantity'] / df['brand_sns_quantity'].shift() - 1
# df['brand_gv_delta'] = df['brand_glance_views'] / df['brand_glance_views'].shift() - 1
# df['brand_search_imp_delta'] = df['brand_total_search_impressions'] / df['brand_total_search_impressions'].shift() - 1
# df['brand_display_imp_delta'] = df['brand_total_display_impressions'] / df['brand_total_display_impressions'].shift() - 1
# df['brand_total_imp_delta'] = df['brand_total_impressions'] / df['brand_total_impressions'].shift() - 1

# df['peer_set_revenue_delta'] = df['peer_set_ordered_revenue'] / df['peer_set_ordered_revenue'].shift() - 1
# df['peer_set_quantity_delta'] = df['peer_set_ordered_quantity'] / df['peer_set_ordered_quantity'].shift() - 1
# df['peer_set_sns_revenue_delta'] = df['peer_set_sns_ordered_revenue'] / df['peer_set_sns_ordered_revenue'].shift() - 1
# df['peer_set_sns_quantity_delta'] = df['peer_set_sns_quantity'] / df['peer_set_sns_quantity'].shift() - 1
# df['peer_set_gv_delta'] = df['peer_set_glance_views'] / df['peer_set_glance_views'].shift() - 1
# df['peer_set_search_imp_delta'] = df['peer_set_total_search_impressions'] / df['peer_set_total_search_impressions'].shift() - 1
# df['peer_set_display_imp_delta'] = df['peer_set_total_display_impressions'] / df['peer_set_total_display_impressions'].shift() - 1
# df['peer_set_total_imp_delta'] = df['peer_set_total_impressions'] / df['peer_set_total_impressions'].shift() - 1

# df.tail()

In [ ]:
# calculate shares
df['brand_revenue_share'] = (df.brand_ordered_revenue / df.peer_set_ordered_revenue).round(3)
df['brand_quantity_share'] = (df.brand_ordered_quantity / df.peer_set_ordered_quantity).round(3)
df['brand_sns_revenue_share'] = (df.brand_sns_ordered_revenue / df.peer_set_sns_ordered_revenue).round(3)
df['brand_sns_quantity_share'] = (df.brand_sns_quantity / df.peer_set_sns_ordered_revenue).round(3)
df['brand_gv_share'] = (df.brand_glance_views / df.peer_set_glance_views).round(3)
df['brand_search_sov'] = (df.brand_total_search_impressions / df.peer_set_total_search_impressions).round(3)
df['brand_display_sov'] = (df.brand_total_display_impressions / df.peer_set_total_display_impressions).round(3)
df['brand_total_sov'] = (df.brand_total_impressions / df.peer_set_total_impressions).round(3)

# calculate CVR
df['brand_cvr'] = (df.brand_ordered_quantity - df.brand_sns_quantity) / df.brand_glance_views
df['peer_set_cvr'] = (df.peer_set_ordered_quantity - df.peer_set_sns_quantity) / df.peer_set_glance_views

df.tail()

In [ ]:
# calculate share deltas (%)
# df['brand_revenue_share_delta'] = df['brand_revenue_share'] / df['brand_revenue_share'].shift() - 1
# df['brand_quantity_share_delta'] = df['brand_quantity_share'] / df['brand_quantity_share'].shift() - 1
# df['brand_sns_revenue_share_delta'] = df['brand_sns_revenue_share'] / df['brand_sns_revenue_share'].shift() - 1
# df['brand_sns_quantity_share_delta'] = df['brand_sns_quantity_share'] / df['brand_sns_quantity_share'].shift() - 1
# df['brand_gv_share_delta'] = df['brand_gv_share'] / df['brand_gv_share'].shift() - 1
# df['brand_search_sov_delta'] = df['brand_search_sov'] / df['brand_search_sov'].shift() - 1
# df['brand_display_sov_delta'] = df['brand_display_sov'] / df['brand_display_sov'].shift() - 1
# df['brand_total_sov_delta'] = df['brand_total_sov'] / df['brand_total_sov'].shift() - 1

# df.tail()

In [ ]:
# calculate indexed deltas
# shares are indexed vs average values during last 4 quarters
df['brand_revenue_share_delta_index'] = df['brand_revenue_share'] / df['brand_revenue_share'].iloc[-4:].mean() * 100
df['brand_quantity_share_delta_index'] = df['brand_quantity_share'] / df['brand_quantity_share'].iloc[-4:].mean() * 100
df['brand_sns_revenue_share_delta_index'] = df['brand_sns_revenue_share'] / df['brand_sns_revenue_share'].iloc[-4:].mean() * 100
df['brand_sns_quantity_share_delta_index'] = df['brand_sns_quantity_share'] / df['brand_sns_quantity_share'].iloc[-4:].mean() * 100
df['brand_gv_share_delta_index'] = df['brand_gv_share'] / df['brand_gv_share'].iloc[-4:].mean() * 100
df['brand_search_sov_delta_index'] = df['brand_search_sov'] / df['brand_search_sov'].iloc[-4:].mean() * 100
df['brand_display_sov_delta_index'] = df['brand_display_sov'] / df['brand_display_sov'].iloc[-4:].mean() * 100
df['brand_total_sov_delta_index'] = df['brand_total_sov'] / df['brand_total_sov'].iloc[-4:].mean() * 100

# indexed cvr
df['brand_cvr_index'] = df['brand_cvr'] / df['peer_set_cvr'] * 100

# SnS contribution
df['brand_sns_revenue_contribution'] = df['brand_sns_ordered_revenue'] / df['brand_ordered_revenue'] * 100
df['peer_set_sns_revenue_contribution'] = df['peer_set_sns_ordered_revenue'] / df['peer_set_ordered_revenue'] * 100

df.tail()

In [ ]:
# reset index
df = df.reset_index()
df = df.fillna(0)

# change last quarter (if needed) to QTD
mask_1 = df.year == current_year
mask_2 = df.quarter.str[1] == current_quarter[1]
df.loc[mask_1 & mask_2,'quarter'] = current_quarter_updated

df

In [ ]:
# export results
export_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
export_file = 'output_quarterly.csv'
df.to_csv(export_path + '/' + export_file, index=False)
del df

#### Monthly

In [ ]:
df = df_monthly\
    .groupby('month_starting_date')\
    .agg({
        # brand
        'brand_ordered_revenue':'sum',
        'brand_ordered_quantity':'sum',
        'brand_sns_ordered_revenue':'sum',
        'brand_sns_quantity':'sum',
        'brand_glance_views':'sum',
        'brand_total_search_impressions':'sum',
        'brand_total_display_impressions':'sum',
        'brand_total_impressions':'sum',
        # peer set
        'peer_set_ordered_revenue':'sum',
        'peer_set_ordered_quantity':'sum',
        'peer_set_sns_ordered_revenue':'sum',
        'peer_set_sns_quantity':'sum',
        'peer_set_glance_views':'sum',
        'peer_set_total_search_impressions':'sum',
        'peer_set_total_display_impressions':'sum',
        'peer_set_total_impressions':'sum'
    })\
    .sort_index()

df.tail()

In [ ]:
# calculate deltas (%)
# df['brand_revenue_delta'] = df['brand_ordered_revenue'] / df['brand_ordered_revenue'].shift() - 1
# df['brand_quantity_delta'] = df['brand_ordered_quantity'] / df['brand_ordered_quantity'].shift() - 1
# df['brand_sns_revenue_delta'] = df['brand_sns_ordered_revenue'] / df['brand_sns_ordered_revenue'].shift() - 1
# df['brand_sns_quantity_delta'] = df['brand_sns_quantity'] / df['brand_sns_quantity'].shift() - 1
# df['brand_gv_delta'] = df['brand_glance_views'] / df['brand_glance_views'].shift() - 1
# df['brand_search_imp_delta'] = df['brand_total_search_impressions'] / df['brand_total_search_impressions'].shift() - 1
# df['brand_display_imp_delta'] = df['brand_total_display_impressions'] / df['brand_total_display_impressions'].shift() - 1
# df['brand_total_imp_delta'] = df['brand_total_impressions'] / df['brand_total_impressions'].shift() - 1

# df['peer_set_revenue_delta'] = df['peer_set_ordered_revenue'] / df['peer_set_ordered_revenue'].shift() - 1
# df['peer_set_quantity_delta'] = df['peer_set_ordered_quantity'] / df['peer_set_ordered_quantity'].shift() - 1
# df['peer_set_sns_revenue_delta'] = df['peer_set_sns_ordered_revenue'] / df['peer_set_sns_ordered_revenue'].shift() - 1
# df['peer_set_sns_quantity_delta'] = df['peer_set_sns_quantity'] / df['peer_set_sns_quantity'].shift() - 1
# df['peer_set_gv_delta'] = df['peer_set_glance_views'] / df['peer_set_glance_views'].shift() - 1
# df['peer_set_search_imp_delta'] = df['peer_set_total_search_impressions'] / df['peer_set_total_search_impressions'].shift() - 1
# df['peer_set_display_imp_delta'] = df['peer_set_total_display_impressions'] / df['peer_set_total_display_impressions'].shift() - 1
# df['peer_set_total_imp_delta'] = df['peer_set_total_impressions'] / df['peer_set_total_impressions'].shift() - 1

# df.tail()

In [ ]:
# calculate shares
df['brand_revenue_share'] = (df.brand_ordered_revenue / df.peer_set_ordered_revenue).round(3)
df['brand_quantity_share'] = (df.brand_ordered_quantity / df.peer_set_ordered_quantity).round(3)
df['brand_sns_revenue_share'] = (df.brand_sns_ordered_revenue / df.peer_set_sns_ordered_revenue).round(3)
df['brand_sns_quantity_share'] = (df.brand_sns_quantity / df.peer_set_sns_ordered_revenue).round(3)
df['brand_gv_share'] = (df.brand_glance_views / df.peer_set_glance_views).round(3)
df['brand_search_sov'] = (df.brand_total_search_impressions / df.peer_set_total_search_impressions).round(3)
df['brand_display_sov'] = (df.brand_total_display_impressions / df.peer_set_total_display_impressions).round(3)
df['brand_total_sov'] = (df.brand_total_impressions / df.peer_set_total_impressions).round(3)

# calculate CVR
df['brand_cvr'] = (df.brand_ordered_quantity - df.brand_sns_quantity) / df.brand_glance_views
df['peer_set_cvr'] = (df.peer_set_ordered_quantity - df.peer_set_sns_quantity) / df.peer_set_glance_views

df.tail()

In [ ]:
# calculate share deltas (%)
# df['brand_revenue_share_delta'] = df['brand_revenue_share'] / df['brand_revenue_share'].shift() - 1
# df['brand_quantity_share_delta'] = df['brand_quantity_share'] / df['brand_quantity_share'].shift() - 1
# df['brand_sns_revenue_share_delta'] = df['brand_sns_revenue_share'] / df['brand_sns_revenue_share'].shift() - 1
# df['brand_sns_quantity_share_delta'] = df['brand_sns_quantity_share'] / df['brand_sns_quantity_share'].shift() - 1
# df['brand_gv_share_delta'] = df['brand_gv_share'] / df['brand_gv_share'].shift() - 1
# df['brand_search_sov_delta'] = df['brand_search_sov'] / df['brand_search_sov'].shift() - 1
# df['brand_display_sov_delta'] = df['brand_display_sov'] / df['brand_display_sov'].shift() - 1
# df['brand_total_sov_delta'] = df['brand_total_sov'] / df['brand_total_sov'].shift() - 1

# df.tail()

In [ ]:
# calculate indexed deltas
# shares are indexed vs average values T12M
df['brand_revenue_share_delta_index'] = df['brand_revenue_share'] / df['brand_revenue_share'].iloc[-12:].mean() * 100
df['brand_quantity_share_delta_index'] = df['brand_quantity_share'] / df['brand_quantity_share'].iloc[-12:].mean() * 100
df['brand_sns_revenue_share_delta_index'] = df['brand_sns_revenue_share'] / df['brand_sns_revenue_share'].iloc[-12:].mean() * 100
df['brand_sns_quantity_share_delta_index'] = df['brand_sns_quantity_share'] / df['brand_sns_quantity_share'].iloc[-12:].mean() * 100
df['brand_gv_share_delta_index'] = df['brand_gv_share'] / df['brand_gv_share'].iloc[-12:].mean() * 100
df['brand_search_sov_delta_index'] = df['brand_search_sov'] / df['brand_search_sov'].iloc[-12:].mean() * 100
df['brand_display_sov_delta_index'] = df['brand_display_sov'] / df['brand_display_sov'].iloc[-12:].mean() * 100
df['brand_total_sov_delta_index'] = df['brand_total_sov'] / df['brand_total_sov'].iloc[-12:].mean() * 100

# indexed cvr
df['brand_cvr_index'] = df['brand_cvr'] / df['peer_set_cvr'] * 100

# SnS contribution
df['brand_sns_revenue_contribution'] = df['brand_sns_ordered_revenue'] / df['brand_ordered_revenue'] * 100
df['peer_set_sns_revenue_contribution'] = df['peer_set_sns_ordered_revenue'] / df['peer_set_ordered_revenue'] * 100

df.tail()

In [ ]:
# reset index
df = df.reset_index()
df = df.fillna(0)
df

In [ ]:
# export results
export_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
export_file = 'output_monthly.csv'
df.to_csv(export_path + '/' + export_file, index=False)
del df

#### Weekly

In [ ]:
df = df_weekly\
    .groupby('week_starting_date')\
    .agg({
        # brand
        'brand_ordered_revenue':'sum',
        'brand_ordered_quantity':'sum',
        'brand_sns_ordered_revenue':'sum',
        'brand_sns_quantity':'sum',
        'brand_glance_views':'sum',
        'brand_total_search_impressions':'sum',
        'brand_total_display_impressions':'sum',
        'brand_total_impressions':'sum',
        # peer set
        'peer_set_ordered_revenue':'sum',
        'peer_set_ordered_quantity':'sum',
        'peer_set_sns_ordered_revenue':'sum',
        'peer_set_sns_quantity':'sum',
        'peer_set_glance_views':'sum',
        'peer_set_total_search_impressions':'sum',
        'peer_set_total_display_impressions':'sum',
        'peer_set_total_impressions':'sum'
    })\
    .sort_index()

df.tail()

In [ ]:
# calculate deltas (%)
# df['brand_revenue_delta'] = df['brand_ordered_revenue'] / df['brand_ordered_revenue'].shift() - 1
# df['brand_quantity_delta'] = df['brand_ordered_quantity'] / df['brand_ordered_quantity'].shift() - 1
# df['brand_sns_revenue_delta'] = df['brand_sns_ordered_revenue'] / df['brand_sns_ordered_revenue'].shift() - 1
# df['brand_sns_quantity_delta'] = df['brand_sns_quantity'] / df['brand_sns_quantity'].shift() - 1
# df['brand_gv_delta'] = df['brand_glance_views'] / df['brand_glance_views'].shift() - 1
# df['brand_search_imp_delta'] = df['brand_total_search_impressions'] / df['brand_total_search_impressions'].shift() - 1
# df['brand_display_imp_delta'] = df['brand_total_display_impressions'] / df['brand_total_display_impressions'].shift() - 1
# df['brand_total_imp_delta'] = df['brand_total_impressions'] / df['brand_total_impressions'].shift() - 1

# df['peer_set_revenue_delta'] = df['peer_set_ordered_revenue'] / df['peer_set_ordered_revenue'].shift() - 1
# df['peer_set_quantity_delta'] = df['peer_set_ordered_quantity'] / df['peer_set_ordered_quantity'].shift() - 1
# df['peer_set_sns_revenue_delta'] = df['peer_set_sns_ordered_revenue'] / df['peer_set_sns_ordered_revenue'].shift() - 1
# df['peer_set_sns_quantity_delta'] = df['peer_set_sns_quantity'] / df['peer_set_sns_quantity'].shift() - 1
# df['peer_set_gv_delta'] = df['peer_set_glance_views'] / df['peer_set_glance_views'].shift() - 1
# df['peer_set_search_imp_delta'] = df['peer_set_total_search_impressions'] / df['peer_set_total_search_impressions'].shift() - 1
# df['peer_set_display_imp_delta'] = df['peer_set_total_display_impressions'] / df['peer_set_total_display_impressions'].shift() - 1
# df['peer_set_total_imp_delta'] = df['peer_set_total_impressions'] / df['peer_set_total_impressions'].shift() - 1

# df.tail()

In [ ]:
# calculate shares
df['brand_revenue_share'] = (df.brand_ordered_revenue / df.peer_set_ordered_revenue).round(3)
df['brand_quantity_share'] = (df.brand_ordered_quantity / df.peer_set_ordered_quantity).round(3)
df['brand_sns_revenue_share'] = (df.brand_sns_ordered_revenue / df.peer_set_sns_ordered_revenue).round(3)
df['brand_sns_quantity_share'] = (df.brand_sns_quantity / df.peer_set_sns_ordered_revenue).round(3)
df['brand_gv_share'] = (df.brand_glance_views / df.peer_set_glance_views).round(3)
df['brand_search_sov'] = (df.brand_total_search_impressions / df.peer_set_total_search_impressions).round(3)
df['brand_display_sov'] = (df.brand_total_display_impressions / df.peer_set_total_display_impressions).round(3)
df['brand_total_sov'] = (df.brand_total_impressions / df.peer_set_total_impressions).round(3)

# calculate CVR
df['brand_cvr'] = (df.brand_ordered_quantity - df.brand_sns_quantity) / df.brand_glance_views
df['peer_set_cvr'] = (df.peer_set_ordered_quantity - df.peer_set_sns_quantity) / df.peer_set_glance_views

df.tail()

In [ ]:
# calculate share deltas (%)
# df['brand_revenue_share_delta'] = df['brand_revenue_share'] / df['brand_revenue_share'].shift() - 1
# df['brand_quantity_share_delta'] = df['brand_quantity_share'] / df['brand_quantity_share'].shift() - 1
# df['brand_sns_revenue_share_delta'] = df['brand_sns_revenue_share'] / df['brand_sns_revenue_share'].shift() - 1
# df['brand_sns_quantity_share_delta'] = df['brand_sns_quantity_share'] / df['brand_sns_quantity_share'].shift() - 1
# df['brand_gv_share_delta'] = df['brand_gv_share'] / df['brand_gv_share'].shift() - 1
# df['brand_search_sov_delta'] = df['brand_search_sov'] / df['brand_search_sov'].shift() - 1
# df['brand_display_sov_delta'] = df['brand_display_sov'] / df['brand_display_sov'].shift() - 1
# df['brand_total_sov_delta'] = df['brand_total_sov'] / df['brand_total_sov'].shift() - 1

# df.tail()

In [ ]:
# calculate indexed deltas
# shares are indexed vs average values T3M
df['brand_revenue_share_delta_index'] = df['brand_revenue_share'] / df['brand_revenue_share'].iloc[-12:].mean() * 100
df['brand_quantity_share_delta_index'] = df['brand_quantity_share'] / df['brand_quantity_share'].iloc[-12:].mean() * 100
df['brand_sns_revenue_share_delta_index'] = df['brand_sns_revenue_share'] / df['brand_sns_revenue_share'].iloc[-12:].mean() * 100
df['brand_sns_quantity_share_delta_index'] = df['brand_sns_quantity_share'] / df['brand_sns_quantity_share'].iloc[-12:].mean() * 100
df['brand_gv_share_delta_index'] = df['brand_gv_share'] / df['brand_gv_share'].iloc[-12:].mean() * 100
df['brand_search_sov_delta_index'] = df['brand_search_sov'] / df['brand_search_sov'].iloc[-12:].mean() * 100
df['brand_display_sov_delta_index'] = df['brand_display_sov'] / df['brand_display_sov'].iloc[-12:].mean() * 100
df['brand_total_sov_delta_index'] = df['brand_total_sov'] / df['brand_total_sov'].iloc[-12:].mean() * 100

# indexed cvr
df['brand_cvr_index'] = df['brand_cvr'] / df['peer_set_cvr'] * 100
# SnS contribution
df['brand_sns_revenue_contribution'] = df['brand_sns_ordered_revenue'] / df['brand_ordered_revenue'] * 100
df['peer_set_sns_revenue_contribution'] = df['peer_set_sns_ordered_revenue'] / df['peer_set_ordered_revenue'] * 100

df.tail()

In [ ]:
# reset index
df = df.reset_index()
df = df.fillna(0)
df.head()

In [ ]:
# export results
export_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
export_file = 'output_weekly.csv'
df.to_csv(export_path + '/' + export_file, index=False)
del df

## 4 PPT production

### 4.1 Import data

In [ ]:
# import
import_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
import_file = 'output_period.csv'
df_yearly = pd.read_csv(import_path + '/' + import_file)

In [ ]:
# import
import_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
import_file = 'output_quarterly.csv'
df_quarterly = pd.read_csv(import_path + '/' + import_file)

In [ ]:
# import
import_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
import_file = 'output_monthly.csv'
df_monthly = pd.read_csv(import_path + '/' + import_file)

In [ ]:
# import
import_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
import_file = 'output_weekly.csv'
df_weekly = pd.read_csv(import_path + '/' + import_file)

### 4.2 Section: PPT backbone

In [ ]:
# build empty dictionary
ppt_dic_s = {}

In [ ]:
pres = Presentation("./resources/PPT_templates/template_basic.pptx")

In [ ]:
# calculate date objects
start_date = datetime.today() - relativedelta(months=12)
start_date = start_date.replace(day=1).strftime('%Y-%m-%d')

end_date = datetime.today().replace(day=1) - timedelta(days=1)
end_date = end_date.strftime('%Y-%m-%d')

# Chart titles, comments, group shapes, tables!
pres.replace({
    "[brand]": input_dic['brand_name'],
    "[locale]": input_dic['local'],
    "[start_date]": start_date,
    "[end_date]": end_date,
    "[category]": input_dic['category_friendly_name'],
    "[root_category]": input_dic['root_browse_node_name'],
    "[category_url]": input_dic['category_url']
})

In [ ]:
# export results
export_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
export_file = input_dic['local']  + '_' + input_dic['brand_name'] + '_BPR_intro_takeaways.pptx'
pres.save(export_path + '/' + export_file)
del pres

### 4.3 Section: reporting slides

#### 4.3.1 Section: sales revenue

In [ ]:
# build empty dictionary
ppt_dic_s = {}

In [ ]:
pres = Presentation("./resources/PPT_templates/template_sales_revenue.pptx")

##### General

In [ ]:
# calculate date objects
start_date = datetime.today() - relativedelta(months=12)
start_date = start_date.replace(day=1).strftime('%Y-%m-%d')

end_date = datetime.today().replace(day=1) - timedelta(days=1)
end_date = end_date.strftime('%Y-%m-%d')

# Chart titles, comments, group shapes, tables!
pres.replace({
    "[brand]": input_dic['brand_name'],
    "[locale]": input_dic['local'],
    "[start_date]": start_date,
    "[end_date]": end_date,
    "[category]": input_dic['category_friendly_name'],
    "[root_category]": input_dic['root_browse_node_name'],
    "[category_url]": input_dic['category_url']
})

##### metrics

In [ ]:
# brand insights
if (df_yearly['brand_ordered_revenue'] > 0).all():
    # we can calculate brand yoy growth rates
    ppt_dic_s['S.01'] = 'increased' if df_yearly['brand_revenue_delta'].iloc[1] > df_yearly['peer_set_revenue_delta'].iloc[1] else 'decreased'
    ppt_dic_s['S.02'] = 'grew' if df_yearly['brand_revenue_delta'].iloc[1] > 0 else 'dropped'
    ppt_dic_s['S.03'] = str(int(df_yearly['brand_revenue_delta'].iloc[1]*100))
    ppt_dic_s['S.04'] = 'grew' if df_yearly['peer_set_revenue_delta'].iloc[1] > 0 else 'dropped'
    ppt_dic_s['S.05'] = str(int(df_yearly['peer_set_revenue_delta'].iloc[1]*100))
    ppt_dic_s['S.06'] = str(round(df_yearly['brand_revenue_share'].iloc[0]*100,2))
    ppt_dic_s['S.07'] = str(round(df_yearly['brand_revenue_share'].iloc[1]*100,2))
    ppt_dic_s['S.08'] = str(int(df_yearly['brand_revenue_share_delta'].iloc[1]*100)) if df_yearly['brand_revenue_share_delta'].iloc[1] != float('inf') else '[remove insight]'   
elif (df_yearly['peer_set_ordered_revenue'] > 0).all():
    # we cannot calculate brand yoy growth, can calculate peer set yoy growth
    ppt_dic_s['S.01'] = '[remove insight]'
    ppt_dic_s['S.02'] = '[remove insight]'
    ppt_dic_s['S.03'] = '[remove insight]'
    ppt_dic_s['S.04'] = 'grew' if df_yearly['peer_set_revenue_delta'].iloc[1] > 0 else 'dropped'
    ppt_dic_s['S.05'] = str(int(df_yearly['peer_set_revenue_delta'].iloc[1]*100))
    ppt_dic_s['S.06'] = '[remove insight]'
    ppt_dic_s['S.07'] = '[remove insight]'
    ppt_dic_s['S.08'] = '[remove insight]'
else:
    # we cannot calculate any yoy growth rates
    ppt_dic_s['S.01'] = '[remove insight]'
    ppt_dic_s['S.02'] = '[remove insight]'
    ppt_dic_s['S.03'] = '[remove insight]'
    ppt_dic_s['S.04'] = '[remove insight]'
    ppt_dic_s['S.05'] = '[remove insight]'
    ppt_dic_s['S.06'] = '[remove insight]'
    ppt_dic_s['S.07'] = '[remove insight]'
    ppt_dic_s['S.08'] = '[remove insight]'


In [ ]:
# replace
pres.replace({
    "[S.01]": ppt_dic_s['S.01'],
    "[S.02]": ppt_dic_s['S.02'],
    "[S.03]": ppt_dic_s['S.03'],
    "[S.04]": ppt_dic_s['S.04'],
    "[S.05]": ppt_dic_s['S.05'],
    "[S.06]": ppt_dic_s['S.06'],
    "[S.07]": ppt_dic_s['S.07'],
    "[S.08]": ppt_dic_s['S.08']
})

##### Chart quarterly

In [ ]:
chart_input = df_quarterly\
                    .loc[:,['quarter','peer_set_ordered_revenue','brand_revenue_share_delta_index']]\
                    .iloc[-4:,:]\
                    .astype({'peer_set_ordered_revenue':'int64','brand_revenue_share_delta_index':'int'})\
                    .rename(
                        columns = {
                            'peer_set_ordered_revenue':'Peer Set Sales Revenue (axis-less)',
                            'brand_revenue_share_delta_index':'Sales Share Index'
                        }
                    )
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=3, 
    chart_name='Chart 1', 
    series_names = ['Peer Set Sales Revenue (axis-less)','Sales Share Index'], 
    df=chart_input, 
    x_column='quarter', 
    y_columns=['Peer Set Sales Revenue (axis-less)','Sales Share Index']
)

##### Chart monthly

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','peer_set_ordered_revenue','brand_revenue_share_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'peer_set_ordered_revenue':'int64',
                             'brand_revenue_share_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'peer_set_ordered_revenue':'Peer Set Sales Revenue (axis-less)',
                            'brand_revenue_share_delta_index':'Sales Share Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=4, 
    chart_name='Chart 1', 
    series_names = ['Peer Set Sales Revenue (axis-less)','Sales Share Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Peer Set Sales Revenue (axis-less)','Sales Share Index']
)

##### Chart weekly

In [ ]:
chart_input = df_weekly\
                    .loc[:,['week_starting_date','peer_set_ordered_revenue','brand_revenue_share_delta_index']]\
                    .iloc[-13:-1,:]\
                    .astype({'week_starting_date':'datetime64[ns]',
                             'peer_set_ordered_revenue':'int64',
                             'brand_revenue_share_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'peer_set_ordered_revenue':'Peer Set Sales Revenue (axis-less)',
                            'brand_revenue_share_delta_index':'Sales Share Index'
                        }
                    )
chart_input['week_starting_date'] = chart_input.week_starting_date.dt.strftime('%m/%d')
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=5, 
    chart_name='Chart 1', 
    series_names = ['Peer Set Sales Revenue (axis-less)','Sales Share Index'], 
    df= chart_input, 
    x_column='week_starting_date', 
    y_columns=['Peer Set Sales Revenue (axis-less)','Sales Share Index']
)

##### Export

In [ ]:
# export results
export_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
export_file = input_dic['local']  + '_' + input_dic['brand_name'] + '_BPR_sales_revenue.pptx'
pres.save(export_path + '/' + export_file)
del pres

#### 4.3.2 Section: glance views

In [ ]:
# build empty dictionary
ppt_dic_g = {}

In [ ]:
pres = Presentation("./resources/PPT_templates/template_glance_views.pptx")

##### General

In [ ]:
# calculate date objects
start_date = datetime.today() - relativedelta(months=12)
start_date = start_date.replace(day=1).strftime('%Y-%m-%d')

end_date = datetime.today().replace(day=1) - timedelta(days=1)
end_date = end_date.strftime('%Y-%m-%d')

# Chart titles, comments, group shapes, tables!
pres.replace({
    "[brand]": input_dic['brand_name'],
    "[locale]": input_dic['local'],
    "[start_date]": start_date,
    "[end_date]": end_date,
    "[category]": input_dic['category_friendly_name'],
    "[root_category]": input_dic['root_browse_node_name'],
    "[category_url]": input_dic['category_url']
})

##### metrics

In [ ]:
# brand insights
if (df_yearly['brand_glance_views'] > 0).all():
    # we can calculate brand yoy growth rates
    ppt_dic_g['G.01'] = 'increased' if df_yearly['brand_gv_delta'].iloc[1] > df_yearly['peer_set_gv_delta'].iloc[1] else 'decreased'
    ppt_dic_g['G.02'] = 'grew' if df_yearly['brand_gv_delta'].iloc[1] > 0 else 'dropped'
    ppt_dic_g['G.03'] = str(int(df_yearly['brand_gv_delta'].iloc[1]*100))
    ppt_dic_g['G.04'] = 'grew' if df_yearly['peer_set_gv_delta'].iloc[1] > 0 else 'dropped'
    ppt_dic_g['G.05'] = str(int(df_yearly['peer_set_gv_delta'].iloc[1]*100))
    ppt_dic_g['G.06'] = str(int(df_yearly['brand_gv_share_delta'].iloc[1]*100)) if df_yearly['brand_gv_share_delta'].iloc[1] != float('inf') else '[remove insight]'   
elif (df_yearly['peer_set_glance_views'] > 0).all():
    # we cannot calculate brand yoy growth, can calculate peer set yoy growth
    ppt_dic_g['G.01'] = '[remove insight]'
    ppt_dic_g['G.02'] = '[remove insight]'
    ppt_dic_g['G.03'] = '[remove insight]'
    ppt_dic_g['G.04'] = 'grew' if df_yearly['peer_set_gv_delta'].iloc[1] > 0 else 'dropped'
    ppt_dic_g['G.05'] = str(int(df_yearly['peer_set_gv_delta'].iloc[1]*100))
    ppt_dic_g['G.06'] = '[remove insight]'
else:
    # we cannot calculate any yoy growth rates
    ppt_dic_g['G.01'] = '[remove insight]'
    ppt_dic_g['G.02'] = '[remove insight]'
    ppt_dic_g['G.03'] = '[remove insight]'
    ppt_dic_g['G.04'] = '[remove insight]'
    ppt_dic_g['G.05'] = '[remove insight]'
    ppt_dic_g['G.06'] = '[remove insight]'


In [ ]:
# replace
pres.replace({
    "[G.01]": ppt_dic_g['G.01'],
    "[G.02]": ppt_dic_g['G.02'],
    "[G.03]": ppt_dic_g['G.03'],
    "[G.04]": ppt_dic_g['G.04'],
    "[G.05]": ppt_dic_g['G.05'],
    "[G.06]": ppt_dic_g['G.06']
})

##### Chart quarterly

In [ ]:
chart_input = df_quarterly\
                    .loc[:,['quarter','peer_set_glance_views','brand_gv_share_delta_index']]\
                    .iloc[-4:,:]\
                    .astype({'peer_set_glance_views':'int64','brand_gv_share_delta_index':'int'})\
                    .rename(
                        columns = {
                            'peer_set_glance_views':'Peer Set Glance Views (axis-less)',
                            'brand_gv_share_delta_index':'Glance Views Share Index'
                        }
                    )
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=2, 
    chart_name='Chart 1', 
    series_names = ['Peer Set Glance Views (axis-less)','Glance Views Share Index'], 
    df=chart_input, 
    x_column='quarter', 
    y_columns=['Peer Set Glance Views (axis-less)','Glance Views Share Index']
)

##### Chart monthly

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','peer_set_glance_views','brand_gv_share_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'peer_set_glance_views':'int64',
                             'brand_gv_share_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'peer_set_glance_views':'Peer Set Glance Views (axis-less)',
                            'brand_gv_share_delta_index':'Glance Views Share Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input


In [ ]:
pres.change_bar_chart(
    slide_number=3, 
    chart_name='Chart 1', 
    series_names = ['Peer Set Glance Views (axis-less)','Glance Views Share Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Peer Set Glance Views (axis-less)','Glance Views Share Index']
)


##### Chart weekly

In [ ]:
chart_input = df_weekly\
                    .loc[:,['week_starting_date','peer_set_glance_views','brand_gv_share_delta_index']]\
                    .iloc[-13:-1,:]\
                    .astype({'week_starting_date':'datetime64[ns]',
                             'peer_set_glance_views':'int64',
                             'brand_gv_share_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'peer_set_glance_views':'Peer Set Glance Views (axis-less)',
                            'brand_gv_share_delta_index':'Glance Views Share Index'
                        }
                    )
chart_input['week_starting_date'] = chart_input.week_starting_date.dt.strftime('%m/%d')
chart_input


In [ ]:
pres.change_bar_chart(
    slide_number=4, 
    chart_name='Chart 1', 
    series_names = ['Peer Set Glance Views (axis-less)','Glance Views Share Index'], 
    df= chart_input, 
    x_column='week_starting_date', 
    y_columns=['Peer Set Glance Views (axis-less)','Glance Views Share Index']
)


##### Export

In [ ]:
# export results
export_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
export_file = input_dic['local']  + '_' + input_dic['brand_name'] + '_BPR_glance_views.pptx'
pres.save(export_path + '/' + export_file)
del pres

#### 4.3.3 Section: display impressions

In [ ]:
# build empty dictionary
ppt_dic_dsp = {}

In [ ]:
pres = Presentation("./resources/PPT_templates/template_display.pptx")

##### General

In [ ]:
# calculate date objects
start_date = datetime.today() - relativedelta(months=12)
start_date = start_date.replace(day=1).strftime('%Y-%m-%d')

end_date = datetime.today().replace(day=1) - timedelta(days=1)
end_date = end_date.strftime('%Y-%m-%d')

# Chart titles, comments, group shapes, tables!
pres.replace({
    "[brand]": input_dic['brand_name'],
    "[locale]": input_dic['local'],
    "[start_date]": start_date,
    "[end_date]": end_date,
    "[category]": input_dic['category_friendly_name'],
    "[root_category]": input_dic['root_browse_node_name'],
    "[category_url]": input_dic['category_url']
})

##### metrics

In [ ]:
df_yearly['brand_display_sov_delta']

In [ ]:
# brand insights
if (df_yearly['brand_total_display_impressions'] > 0).all():
    # we can calculate brand yoy growth rates    
    ppt_dic_dsp['D.01'] = 'increased' if df_yearly['brand_display_imp_delta'].iloc[1] > df_yearly['peer_set_display_imp_delta'].iloc[1] else 'decreased'
    ppt_dic_dsp['D.02'] = 'grew' if df_yearly['brand_display_imp_delta'].iloc[1] > 0 else 'dropped'
    ppt_dic_dsp['D.03'] = str(int(df_yearly['brand_display_imp_delta'].iloc[1]*100))
    ppt_dic_dsp['D.04'] = 'grew' if df_yearly['peer_set_display_imp_delta'].iloc[1] > 0 else 'dropped'
    ppt_dic_dsp['D.05'] = str(int(df_yearly['peer_set_display_imp_delta'].iloc[1]*100))
    ppt_dic_dsp['D.06'] = str(int(df_yearly['brand_display_sov_delta'].iloc[1]*100)) if df_yearly['brand_display_sov_delta'].iloc[1] != float('inf') else '[remove insight]'   
    
elif (df_yearly['peer_set_total_display_impressions'] > 0).all():
    # we cannot calculate brand yoy growth, can calculate peer set yoy growth
    ppt_dic_dsp['D.01'] = '[remove insight]'
    ppt_dic_dsp['D.02'] = '[remove insight]'
    ppt_dic_dsp['D.03'] = '[remove insight]'
    ppt_dic_dsp['D.04'] = 'grew' if df_yearly['peer_set_display_imp_delta'].iloc[1] > 0 else 'dropped'
    ppt_dic_dsp['D.05'] = str(int(df_yearly['peer_set_display_imp_delta'].iloc[1]*100))
    ppt_dic_dsp['D.06'] = '[remove insight]'    
    
else:
    # we cannot calculate any yoy growth rates
    ppt_dic_dsp['D.01'] = '[remove insight]'
    ppt_dic_dsp['D.02'] = '[remove insight]'
    ppt_dic_dsp['D.03'] = '[remove insight]'
    ppt_dic_dsp['D.04'] = '[remove insight]'
    ppt_dic_dsp['D.05'] = '[remove insight]'
    ppt_dic_dsp['D.06'] = '[remove insight]'


In [ ]:
# replace
pres.replace({
    "[D.01]": ppt_dic_dsp['D.01'],
    "[D.02]": ppt_dic_dsp['D.02'],
    "[D.03]": ppt_dic_dsp['D.03'],
    "[D.04]": ppt_dic_dsp['D.04'],
    "[D.05]": ppt_dic_dsp['D.05'],
    "[D.06]": ppt_dic_dsp['D.06']
})

##### Chart quarterly

In [ ]:
chart_input = df_quarterly\
                    .loc[:,['quarter','peer_set_total_display_impressions','brand_display_sov_delta_index']]\
                    .iloc[-4:,:]\
                    .astype({'peer_set_total_display_impressions':'int64','brand_display_sov_delta_index':'int'})\
                    .rename(
                        columns = {
                            'peer_set_total_display_impressions':'Peer Set Display Impressions (axis-less)',
                            'brand_display_sov_delta_index':'Display SOV Index'
                        }
                    )
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=2, 
    chart_name='Chart 1', 
    series_names = ['Peer Set Display Impressions (axis-less)','Display SOV Index'], 
    df=chart_input, 
    x_column='quarter', 
    y_columns=['Peer Set Display Impressions (axis-less)','Display SOV Index']
)

##### Chart monthly

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','peer_set_total_display_impressions','brand_display_sov_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'peer_set_total_display_impressions':'int64',
                             'brand_display_sov_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'peer_set_total_display_impressions':'Peer Set Display Impressions (axis-less)',
                            'brand_display_sov_delta_index':'Display SOV Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input


In [ ]:
pres.change_bar_chart(
    slide_number=3, 
    chart_name='Chart 1', 
    series_names = ['Peer Set Display Impressions (axis-less)','Display SOV Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Peer Set Display Impressions (axis-less)','Display SOV Index']
)


##### Chart weekly

In [ ]:
chart_input = df_weekly\
                    .loc[:,['week_starting_date','peer_set_total_display_impressions','brand_display_sov_delta_index']]\
                    .iloc[-13:-1,:]\
                    .astype({'week_starting_date':'datetime64[ns]',
                             'peer_set_total_display_impressions':'int64',
                             'brand_display_sov_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'peer_set_total_display_impressions':'Peer Set Display Impressions (axis-less)',
                            'brand_display_sov_delta_index':'Display SOV Index'
                        }
                    )
chart_input['week_starting_date'] = chart_input.week_starting_date.dt.strftime('%m/%d')
chart_input


In [ ]:
pres.change_bar_chart(
    slide_number=4, 
    chart_name='Chart 1', 
    series_names = ['Peer Set Display Impressions (axis-less)','Display SOV Index'], 
    df= chart_input, 
    x_column='week_starting_date', 
    y_columns=['Peer Set Display Impressions (axis-less)','Display SOV Index']
)


##### Export

In [ ]:
# export results
export_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
export_file = input_dic['local']  + '_' + input_dic['brand_name'] + '_BPR_display.pptx'
pres.save(export_path + '/' + export_file)
del pres

#### 4.3.4 Section: search impressions

In [ ]:
# build empty dictionary
ppt_dic_srch = {}

In [ ]:
pres = Presentation("./resources/PPT_templates/template_search.pptx")

##### General

In [ ]:
# calculate date objects
start_date = datetime.today() - relativedelta(months=12)
start_date = start_date.replace(day=1).strftime('%Y-%m-%d')

end_date = datetime.today().replace(day=1) - timedelta(days=1)
end_date = end_date.strftime('%Y-%m-%d')

# Chart titles, comments, group shapes, tables!
pres.replace({
    "[brand]": input_dic['brand_name'],
    "[locale]": input_dic['local'],
    "[start_date]": start_date,
    "[end_date]": end_date,
    "[category]": input_dic['category_friendly_name'],
    "[root_category]": input_dic['root_browse_node_name'],
    "[category_url]": input_dic['category_url']
})

##### metrics

In [ ]:
# brand insights
if (df_yearly['brand_total_search_impressions'] > 0).all():
    # we can calculate brand yoy growth rates
    ppt_dic_srch['SS.01'] = 'increased' if df_yearly['brand_search_imp_delta'].iloc[1] > df_yearly['peer_set_search_imp_delta'].iloc[1] else 'decreased'
    ppt_dic_srch['SS.02'] = 'grew' if df_yearly['brand_search_imp_delta'].iloc[1] > 0 else 'dropped'
    ppt_dic_srch['SS.03'] = str(int(df_yearly['brand_search_imp_delta'].iloc[1]*100))
    ppt_dic_srch['SS.04'] = 'grew' if df_yearly['peer_set_search_imp_delta'].iloc[1] > 0 else 'dropped'
    ppt_dic_srch['SS.05'] = str(int(df_yearly['peer_set_search_imp_delta'].iloc[1]*100))
    ppt_dic_srch['SS.06'] = str(int(df_yearly['brand_search_sov_delta'].iloc[1]*100)) if df_yearly['brand_search_sov_delta'].iloc[1] != float('inf') else '[remove insight]'
    
elif (df_yearly['peer_set_total_search_impressions'] > 0).all():
    # we cannot calculate brand yoy growth, can calculate peer set yoy growth
    ppt_dic_srch['SS.01'] = '[remove insight]'
    ppt_dic_srch['SS.02'] = '[remove insight]'
    ppt_dic_srch['SS.03'] = str(int(df_yearly['peer_set_search_imp_delta'].iloc[1]*100))
    ppt_dic_srch['SS.04'] = 'increased' if df_yearly['brand_search_imp_delta'].iloc[1] > df_yearly['peer_set_search_imp_delta'].iloc[1] else 'decreased'
    ppt_dic_srch['SS.05'] = '[remove insight]'
    ppt_dic_srch['SS.06'] = '[remove insight]'    
    
else:
    # we cannot calculate any yoy growth rates
    ppt_dic_srch['SS.01'] = '[remove insight]'
    ppt_dic_srch['SS.02'] = '[remove insight]'
    ppt_dic_srch['SS.03'] = '[remove insight]'
    ppt_dic_srch['SS.04'] = '[remove insight]'
    ppt_dic_srch['SS.05'] = '[remove insight]'
    ppt_dic_srch['SS.06'] = '[remove insight]'


In [ ]:
# replace
pres.replace({
    "[SS.01]": ppt_dic_srch['SS.01'],
    "[SS.02]": ppt_dic_srch['SS.02'],
    "[SS.03]": ppt_dic_srch['SS.03'],
    "[SS.04]": ppt_dic_srch['SS.04'],
    "[SS.05]": ppt_dic_srch['SS.05'],
    "[SS.06]": ppt_dic_srch['SS.06']
})

##### Chart quarterly

In [ ]:
chart_input = df_quarterly\
                    .loc[:,['quarter','peer_set_total_search_impressions','brand_search_sov_delta_index']]\
                    .iloc[-4:,:]\
                    .astype({'peer_set_total_search_impressions':'int64','brand_search_sov_delta_index':'int'})\
                    .rename(
                        columns = {
                            'peer_set_total_search_impressions':'Peer Set Search Impressions (axis-less)',
                            'brand_search_sov_delta_index':'Search SOV Index'
                        }
                    )
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=2, 
    chart_name='Chart 1', 
    series_names = ['Peer Set Search Impressions (axis-less)','Search SOV Index'], 
    df=chart_input, 
    x_column='quarter', 
    y_columns=['Peer Set Search Impressions (axis-less)','Search SOV Index']
)

##### Chart monthly

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','peer_set_total_search_impressions','brand_search_sov_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'peer_set_total_search_impressions':'int64',
                             'brand_search_sov_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'peer_set_total_search_impressions':'Peer Set Search Impressions (axis-less)',
                            'brand_search_sov_delta_index':'Search SOV Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input


In [ ]:
pres.change_bar_chart(
    slide_number=3, 
    chart_name='Chart 1', 
    series_names = ['Peer Set Search Impressions (axis-less)','Search SOV Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Peer Set Search Impressions (axis-less)','Search SOV Index']
)


##### Chart weekly

In [ ]:
chart_input = df_weekly\
                    .loc[:,['week_starting_date','peer_set_total_search_impressions','brand_search_sov_delta_index']]\
                    .iloc[-13:-1,:]\
                    .astype({'week_starting_date':'datetime64[ns]',
                             'peer_set_total_search_impressions':'int64',
                             'brand_search_sov_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'peer_set_total_search_impressions':'Peer Set Search Impressions (axis-less)',
                            'brand_search_sov_delta_index':'Search SOV Index'
                        }
                    )
chart_input['week_starting_date'] = chart_input.week_starting_date.dt.strftime('%m/%d')
chart_input


In [ ]:
pres.change_bar_chart(
    slide_number=4, 
    chart_name='Chart 1', 
    series_names = ['Peer Set Search Impressions (axis-less)','Search SOV Index'], 
    df= chart_input, 
    x_column='week_starting_date', 
    y_columns=['Peer Set Search Impressions (axis-less)','Search SOV Index']
)


##### Export

In [ ]:
# export results
export_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
export_file = input_dic['local']  + '_' + input_dic['brand_name'] + '_BPR_search.pptx'
pres.save(export_path + '/' + export_file)
del pres

#### 4.3.5 Section: conversion

In [ ]:
pres = Presentation("./resources/PPT_templates/template_conversion_rate.pptx")

##### General

In [ ]:
# calculate date objects
start_date = datetime.today() - relativedelta(months=12)
start_date = start_date.replace(day=1).strftime('%Y-%m-%d')

end_date = datetime.today().replace(day=1) - timedelta(days=1)
end_date = end_date.strftime('%Y-%m-%d')

# Chart titles, comments, group shapes, tables!
pres.replace({
    "[brand]": input_dic['brand_name'],
    "[locale]": input_dic['local'],
    "[start_date]": start_date,
    "[end_date]": end_date,
    "[category]": input_dic['category_friendly_name'],
    "[root_category]": input_dic['root_browse_node_name'],
    "[category_url]": input_dic['category_url']
})

##### metrics

In [ ]:
# brand insights
if (df_yearly['brand_cvr'] > 0).all():
    # we can calculate brand yoy growth rates
    ppt_dic_srch['CV.01'] = 'higher' if df_yearly['brand_cvr'].iloc[1] > df_yearly['peer_set_cvr'].iloc[1] else 'lower'
    ppt_dic_srch['CV.02'] = 'increased' if df_yearly['brand_cvr'].iloc[1] > df_yearly['brand_cvr'].iloc[0] else 'decreased'
    ppt_dic_srch['CV.03'] = str(int(df_yearly['brand_cvr'].iloc[0]*100))
    ppt_dic_srch['CV.04'] = str(int(df_yearly['brand_cvr'].iloc[1]*100))
    ppt_dic_srch['CV.05'] = str(abs(round((df_yearly['brand_cvr'].iloc[1]/df_yearly['peer_set_cvr'].iloc[1]-1)*100)))
    
else:
    # we cannot calculate any yoy growth rates
    ppt_dic_srch['CV.01'] = '[remove insight]'
    ppt_dic_srch['CV.02'] = '[remove insight]'
    ppt_dic_srch['CV.03'] = '[remove insight]'
    ppt_dic_srch['CV.04'] = '[remove insight]'
    ppt_dic_srch['CV.05'] = '[remove insight]'
    ppt_dic_srch['CV.06'] = '[remove insight]'


In [ ]:
# replace
pres.replace({
    "[CV.01]": ppt_dic_srch['CV.01'],
    "[CV.02]": ppt_dic_srch['CV.02'],
    "[CV.03]": ppt_dic_srch['CV.03'],
    "[CV.04]": ppt_dic_srch['CV.04'],
    "[CV.05]": ppt_dic_srch['CV.05']
})

##### Chart quarterly

In [ ]:
chart_input = df_quarterly\
                    .loc[:,['quarter','brand_cvr','brand_cvr_index']]\
                    .iloc[-4:,:]\
                    .astype({'brand_cvr_index':'int'})\
                    .rename(
                        columns = {
                            'brand_cvr':'Brand CVR (left)',
                            'brand_cvr_index':'CVR Index'
                        }
                    )
chart_input


In [ ]:
pres.change_bar_chart(
    slide_number=2, 
    chart_name='Chart 1', 
    series_names = ['Brand CVR (left)','CVR Index'], 
    df=chart_input, 
    x_column='quarter', 
    y_columns=['Brand CVR (left)','CVR Index']
)

##### Chart monthly

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','brand_cvr','brand_cvr_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'brand_cvr_index':'int'
                            })\
                    .rename(
                        columns = {
                            'brand_cvr':'Brand CVR (left)',
                            'brand_cvr_index':'CVR Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input


In [ ]:
pres.change_bar_chart(
    slide_number=3, 
    chart_name='Chart 1', 
    series_names = ['Brand CVR (left)','CVR Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Brand CVR (left)','CVR Index']
)


##### Chart weekly

In [ ]:
chart_input = df_weekly\
                    .loc[:,['week_starting_date','brand_cvr','brand_cvr_index']]\
                    .iloc[-13:-1,:]\
                    .astype({'week_starting_date':'datetime64[ns]',
                             'brand_cvr_index':'int'
                            })\
                    .rename(
                        columns = {
                            'brand_cvr':'Brand CVR (left)',
                            'brand_cvr_index':'CVR Index'
                        }
                    )
chart_input['week_starting_date'] = chart_input.week_starting_date.dt.strftime('%m/%d')
chart_input


In [ ]:
pres.change_bar_chart(
    slide_number=4, 
    chart_name='Chart 1', 
    series_names = ['Brand CVR (left)','CVR Index'],
    df= chart_input, 
    x_column='week_starting_date', 
    y_columns=['Brand CVR (left)','CVR Index']
)


##### Export

In [ ]:
# export results
export_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
export_file = input_dic['local']  + '_' + input_dic['brand_name'] + '_BPR_conversion.pptx'
pres.save(export_path + '/' + export_file)
del pres

### 4.4 Section: insights

In [ ]:
# build empty dictionary
# ppt_dic_overlap = {}

In [ ]:
pres = Presentation("./resources/PPT_templates/template_insights.pptx")

##### General

In [ ]:
# calculate date objects
start_date = datetime.today() - relativedelta(months=12)
start_date = start_date.replace(day=1).strftime('%Y-%m-%d')

end_date = datetime.today().replace(day=1) - timedelta(days=1)
end_date = end_date.strftime('%Y-%m-%d')

# Chart titles, comments, group shapes, tables!
pres.replace({
    "[brand]": input_dic['brand_name'],
    "[locale]": input_dic['local'],
    "[start_date]": start_date,
    "[end_date]": end_date,
    "[category]": input_dic['category_friendly_name'],
    "[root_category]": input_dic['root_browse_node_name'],
    "[category_url]": input_dic['category_url']
})

##### Chart monthly: sales vs glance views

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','brand_revenue_share_delta_index','brand_gv_share_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'brand_revenue_share_delta_index':'int64',
                             'brand_gv_share_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'brand_revenue_share_delta_index':'Sales Share Index',
                            'brand_gv_share_delta_index':'Glance Views Share Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=1, 
    chart_name='Chart 1', 
    series_names = ['Sales Share Index','Glance Views Share Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Sales Share Index','Glance Views Share Index']
)


##### Chart monthly: glance views vs search/display

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','brand_gv_share_delta_index','brand_display_sov_delta_index','brand_search_sov_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'brand_gv_share_delta_index':'int',
                             'brand_display_sov_delta_index':'int',
                             'brand_search_sov_delta_index':'int'                             
                            })\
                    .rename(
                        columns = {
                            'brand_gv_share_delta_index':'Glance Views Share Index',
                            'brand_display_sov_delta_index':'Display SOV Index',
                            'brand_search_sov_delta_index':'Search SOV Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=2, 
    chart_name='Chart 1', 
    series_names = ['Glance Views Share Index','Display SOV Index','Search SOV Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Glance Views Share Index','Display SOV Index','Search SOV Index']
)


##### Chart monthly: sales vs search/display

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','brand_revenue_share_delta_index','brand_display_sov_delta_index','brand_search_sov_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'brand_revenue_share_delta_index':'int',
                             'brand_display_sov_delta_index':'int',
                             'brand_search_sov_delta_index':'int'                             
                            })\
                    .rename(
                        columns = {
                            'brand_revenue_share_delta_index':'Sales Share Index',
                            'brand_display_sov_delta_index':'Display SOV Index',
                            'brand_search_sov_delta_index':'Search SOV Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=3, 
    chart_name='Chart 1', 
    series_names = ['Sales Share Index','Display SOV Index','Search SOV Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Sales Share Index','Display SOV Index','Search SOV Index']
)


##### Export

In [ ]:
# export results
export_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
export_file = input_dic['local']  + '_' + input_dic['brand_name'] + '_BPR_overlap.pptx'
pres.save(export_path + '/' + export_file)
del pres

### 4.5 Section: A4 (pager) report generation

#### 4.5.1 Section: brand performance report - version 1

In [ ]:
# build empty dictionary
# ppt_dic_s = {}

In [ ]:
pres = Presentation("./resources/PPT_templates/template_brand_performance_report_v1.pptx")

##### General

In [ ]:
# calculate date objects
start_date = datetime.today() - relativedelta(months=12)
start_date = start_date.replace(day=1).strftime('%Y-%m-%d')

end_date = datetime.today().replace(day=1) - timedelta(days=1)
end_date = end_date.strftime('%Y-%m-%d')

# Chart titles, comments, group shapes, tables!
pres.replace({
    "[brand]": input_dic['brand_name'],
    "[locale]": input_dic['local'],
    "[start_date]": start_date,
    "[end_date]": end_date,
    "[category]": input_dic['category_friendly_name'],
    "[root_category]": input_dic['root_browse_node_name'],
    "[category_url]": input_dic['category_url']
})

##### metrics

In [ ]:
# brand insights
if (df_yearly['brand_ordered_revenue'] > 0).all():
    # we can calculate brand yoy growth rates
    ppt_dic_s['S.01'] = 'increased' if df_yearly['brand_revenue_delta'].iloc[1] > df_yearly['peer_set_revenue_delta'].iloc[1] else 'decreased'
    ppt_dic_s['S.02'] = 'grew' if df_yearly['brand_revenue_delta'].iloc[1] > 0 else 'dropped'
    ppt_dic_s['S.03'] = str(int(df_yearly['brand_revenue_delta'].iloc[1]*100))
    ppt_dic_s['S.04'] = 'grew' if df_yearly['peer_set_revenue_delta'].iloc[1] > 0 else 'dropped'
    ppt_dic_s['S.05'] = str(int(df_yearly['peer_set_revenue_delta'].iloc[1]*100))
    ppt_dic_s['S.06'] = str(round(df_yearly['brand_revenue_share'].iloc[0]*100,2))
    ppt_dic_s['S.07'] = str(round(df_yearly['brand_revenue_share'].iloc[1]*100,2))
    ppt_dic_s['S.08'] = str(int(df_yearly['brand_revenue_share_delta'].iloc[1]*100)) if df_yearly['brand_revenue_share_delta'].iloc[1] != float('inf') else '[remove insight]'   
elif (df_yearly['peer_set_ordered_revenue'] > 0).all():
    # we cannot calculate brand yoy growth, can calculate peer set yoy growth
    ppt_dic_s['S.01'] = '[remove insight]'
    ppt_dic_s['S.02'] = '[remove insight]'
    ppt_dic_s['S.03'] = '[remove insight]'
    ppt_dic_s['S.04'] = 'grew' if df_yearly['peer_set_revenue_delta'].iloc[1] > 0 else 'dropped'
    ppt_dic_s['S.05'] = str(int(df_yearly['peer_set_revenue_delta'].iloc[1]*100))
    ppt_dic_s['S.06'] = '[remove insight]'
    ppt_dic_s['S.07'] = '[remove insight]'
    ppt_dic_s['S.08'] = '[remove insight]'
else:
    # we cannot calculate any yoy growth rates
    ppt_dic_s['S.01'] = '[remove insight]'
    ppt_dic_s['S.02'] = '[remove insight]'
    ppt_dic_s['S.03'] = '[remove insight]'
    ppt_dic_s['S.04'] = '[remove insight]'
    ppt_dic_s['S.05'] = '[remove insight]'
    ppt_dic_s['S.06'] = '[remove insight]'
    ppt_dic_s['S.07'] = '[remove insight]'
    ppt_dic_s['S.08'] = '[remove insight]'


In [ ]:
# replace
pres.replace({
    "[S.01]": ppt_dic_s['S.01'],
    "[S.02]": ppt_dic_s['S.02'],
    "[S.03]": ppt_dic_s['S.03'],
    "[S.04]": ppt_dic_s['S.04'],
    "[S.05]": ppt_dic_s['S.05'],
    "[S.06]": ppt_dic_s['S.06'],
    "[S.07]": ppt_dic_s['S.07'],
    "[S.08]": ppt_dic_s['S.08']
})

#### charts sales

##### Chart quarterly

In [ ]:
chart_input = df_quarterly\
                    .loc[:,['quarter','peer_set_ordered_revenue','brand_revenue_share_delta_index']]\
                    .iloc[-4:,:]\
                    .astype({'peer_set_ordered_revenue':'int64','brand_revenue_share_delta_index':'int'})\
                    .rename(
                        columns = {
                            'peer_set_ordered_revenue':'Peer Set Sales Revenue (axis-less)',
                            'brand_revenue_share_delta_index':'Sales Share Index'
                        }
                    )
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=2, 
    chart_name='sales_chart', 
    series_names = ['Peer Set Sales Revenue (axis-less)','Sales Share Index'], 
    df=chart_input, 
    x_column='quarter', 
    y_columns=['Peer Set Sales Revenue (axis-less)','Sales Share Index']
)

##### Chart monthly

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','peer_set_ordered_revenue','brand_revenue_share_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'peer_set_ordered_revenue':'int64',
                             'brand_revenue_share_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'peer_set_ordered_revenue':'Peer Set Sales Revenue (axis-less)',
                            'brand_revenue_share_delta_index':'Sales Share Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=1, 
    chart_name='sales_chart', 
    series_names = ['Peer Set Sales Revenue (axis-less)','Sales Share Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Peer Set Sales Revenue (axis-less)','Sales Share Index']
)

#### charts glance views

##### Chart quarterly

In [ ]:
chart_input = df_quarterly\
                    .loc[:,['quarter','peer_set_glance_views','brand_gv_share_delta_index']]\
                    .iloc[-4:,:]\
                    .astype({'peer_set_glance_views':'int64','brand_gv_share_delta_index':'int'})\
                    .rename(
                        columns = {
                            'peer_set_glance_views':'Peer Set Glance Views (axis-less)',
                            'brand_gv_share_delta_index':'Glance Views Share Index'
                        }
                    )
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=2, 
    chart_name='gvs_chart', 
    series_names = ['Peer Set Glance Views (axis-less)','Glance Views Share Index'], 
    df=chart_input, 
    x_column='quarter', 
    y_columns=['Peer Set Glance Views (axis-less)','Glance Views Share Index']
)

##### Chart monthly

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','peer_set_glance_views','brand_gv_share_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'peer_set_glance_views':'int64',
                             'brand_gv_share_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'peer_set_glance_views':'Peer Set Glance Views (axis-less)',
                            'brand_gv_share_delta_index':'Glance Views Share Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input


In [ ]:
pres.change_bar_chart(
    slide_number=1, 
    chart_name='gvs_chart', 
    series_names = ['Peer Set Glance Views (axis-less)','Glance Views Share Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Peer Set Glance Views (axis-less)','Glance Views Share Index']
)


#### charts display

##### Chart quarterly

In [ ]:
chart_input = df_quarterly\
                    .loc[:,['quarter','peer_set_total_display_impressions','brand_display_sov_delta_index']]\
                    .iloc[-4:,:]\
                    .astype({'peer_set_total_display_impressions':'int64','brand_display_sov_delta_index':'int'})\
                    .rename(
                        columns = {
                            'peer_set_total_display_impressions':'Peer Set Display Impressions (axis-less)',
                            'brand_display_sov_delta_index':'Display SOV Index'
                        }
                    )
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=2, 
    chart_name='display_chart', 
    series_names = ['Peer Set Display Impressions (axis-less)','Display SOV Index'], 
    df=chart_input, 
    x_column='quarter', 
    y_columns=['Peer Set Display Impressions (axis-less)','Display SOV Index']
)

##### Chart monthly

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','peer_set_total_display_impressions','brand_display_sov_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'peer_set_total_display_impressions':'int64',
                             'brand_display_sov_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'peer_set_total_display_impressions':'Peer Set Display Impressions (axis-less)',
                            'brand_display_sov_delta_index':'Display SOV Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input


In [ ]:
pres.change_bar_chart(
    slide_number=1, 
    chart_name='display_chart', 
    series_names = ['Peer Set Display Impressions (axis-less)','Display SOV Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Peer Set Display Impressions (axis-less)','Display SOV Index']
)


#### charts search

##### Chart quarterly

In [ ]:
chart_input = df_quarterly\
                    .loc[:,['quarter','peer_set_total_search_impressions','brand_search_sov_delta_index']]\
                    .iloc[-4:,:]\
                    .astype({'peer_set_total_search_impressions':'int64','brand_search_sov_delta_index':'int'})\
                    .rename(
                        columns = {
                            'peer_set_total_search_impressions':'Peer Set Search Impressions (axis-less)',
                            'brand_search_sov_delta_index':'Search SOV Index'
                        }
                    )
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=2, 
    chart_name='search_chart', 
    series_names = ['Peer Set Search Impressions (axis-less)','Search SOV Index'], 
    df=chart_input, 
    x_column='quarter', 
    y_columns=['Peer Set Search Impressions (axis-less)','Search SOV Index']
)

##### Chart monthly

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','peer_set_total_search_impressions','brand_search_sov_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'peer_set_total_search_impressions':'int64',
                             'brand_search_sov_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'peer_set_total_search_impressions':'Peer Set Search Impressions (axis-less)',
                            'brand_search_sov_delta_index':'Search SOV Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input


In [ ]:
pres.change_bar_chart(
    slide_number=1, 
    chart_name='search_chart', 
    series_names = ['Peer Set Search Impressions (axis-less)','Search SOV Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Peer Set Search Impressions (axis-less)','Search SOV Index']
)


#### charts overlap 1

##### Chart monthly: sales vs glance views

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','brand_revenue_share_delta_index','brand_gv_share_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'brand_revenue_share_delta_index':'int64',
                             'brand_gv_share_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'brand_revenue_share_delta_index':'Sales Share Index',
                            'brand_gv_share_delta_index':'Glance Views Share Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=1, 
    chart_name='overlap_chart_1', 
    series_names = ['Sales Share Index','Glance Views Share Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Sales Share Index','Glance Views Share Index']
)


#### charts overlap 2

##### Chart monthly: glance views vs search/display

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','brand_gv_share_delta_index','brand_display_sov_delta_index','brand_search_sov_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'brand_gv_share_delta_index':'int',
                             'brand_display_sov_delta_index':'int',
                             'brand_search_sov_delta_index':'int'                             
                            })\
                    .rename(
                        columns = {
                            'brand_gv_share_delta_index':'Glance Views Share Index',
                            'brand_display_sov_delta_index':'Display SOV Index',
                            'brand_search_sov_delta_index':'Search SOV Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=1, 
    chart_name='overlap_chart_2', 
    series_names = ['Glance Views Share Index','Display SOV Index','Search SOV Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Glance Views Share Index','Display SOV Index','Search SOV Index']
)


#### Export

In [ ]:
# export results
export_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
export_file = input_dic['local']  + '_' + input_dic['brand_name'] + '_brand_performance_report_v1.pptx'
pres.save(export_path + '/' + export_file)
del pres

#### 4.5.2 Section: brand performance report - version 2

In [ ]:
# build empty dictionary
# ppt_dic_s = {}

In [ ]:
pres = Presentation("./resources/PPT_templates/template_brand_performance_report_v2.pptx")

##### General

In [ ]:
# calculate date objects
start_date = datetime.today() - relativedelta(months=12)
start_date = start_date.replace(day=1).strftime('%Y-%m-%d')

end_date = datetime.today().replace(day=1) - timedelta(days=1)
end_date = end_date.strftime('%Y-%m-%d')

# Chart titles, comments, group shapes, tables!
pres.replace({
    "[brand]": input_dic['brand_name'],
    "[locale]": input_dic['local'],
    "[start_date]": start_date,
    "[end_date]": end_date,
    "[category]": input_dic['category_friendly_name'],
    "[root_category]": input_dic['root_browse_node_name'],
    "[category_url]": input_dic['category_url']
})

##### metrics

In [ ]:
# brand insights
if (df_yearly['brand_ordered_revenue'] > 0).all():
    # we can calculate brand yoy growth rates
    ppt_dic_s['S.01'] = 'increased' if df_yearly['brand_revenue_delta'].iloc[1] > df_yearly['peer_set_revenue_delta'].iloc[1] else 'decreased'
    ppt_dic_s['S.02'] = 'grew' if df_yearly['brand_revenue_delta'].iloc[1] > 0 else 'dropped'
    ppt_dic_s['S.03'] = str(int(df_yearly['brand_revenue_delta'].iloc[1]*100))
    ppt_dic_s['S.04'] = 'grew' if df_yearly['peer_set_revenue_delta'].iloc[1] > 0 else 'dropped'
    ppt_dic_s['S.05'] = str(int(df_yearly['peer_set_revenue_delta'].iloc[1]*100))
    ppt_dic_s['S.06'] = str(round(df_yearly['brand_revenue_share'].iloc[0]*100,2))
    ppt_dic_s['S.07'] = str(round(df_yearly['brand_revenue_share'].iloc[1]*100,2))
    ppt_dic_s['S.08'] = str(int(df_yearly['brand_revenue_share_delta'].iloc[1]*100)) if df_yearly['brand_revenue_share_delta'].iloc[1] != float('inf') else '[remove insight]'   
elif (df_yearly['peer_set_ordered_revenue'] > 0).all():
    # we cannot calculate brand yoy growth, can calculate peer set yoy growth
    ppt_dic_s['S.01'] = '[remove insight]'
    ppt_dic_s['S.02'] = '[remove insight]'
    ppt_dic_s['S.03'] = '[remove insight]'
    ppt_dic_s['S.04'] = 'grew' if df_yearly['peer_set_revenue_delta'].iloc[1] > 0 else 'dropped'
    ppt_dic_s['S.05'] = str(int(df_yearly['peer_set_revenue_delta'].iloc[1]*100))
    ppt_dic_s['S.06'] = '[remove insight]'
    ppt_dic_s['S.07'] = '[remove insight]'
    ppt_dic_s['S.08'] = '[remove insight]'
else:
    # we cannot calculate any yoy growth rates
    ppt_dic_s['S.01'] = '[remove insight]'
    ppt_dic_s['S.02'] = '[remove insight]'
    ppt_dic_s['S.03'] = '[remove insight]'
    ppt_dic_s['S.04'] = '[remove insight]'
    ppt_dic_s['S.05'] = '[remove insight]'
    ppt_dic_s['S.06'] = '[remove insight]'
    ppt_dic_s['S.07'] = '[remove insight]'
    ppt_dic_s['S.08'] = '[remove insight]'


In [ ]:
# replace
pres.replace({
    "[S.01]": ppt_dic_s['S.01'],
    "[S.02]": ppt_dic_s['S.02'],
    "[S.03]": ppt_dic_s['S.03'],
    "[S.04]": ppt_dic_s['S.04'],
    "[S.05]": ppt_dic_s['S.05'],
    "[S.06]": ppt_dic_s['S.06'],
    "[S.07]": ppt_dic_s['S.07'],
    "[S.08]": ppt_dic_s['S.08']
})

#### charts sales

##### Chart quarterly

In [ ]:
chart_input = df_quarterly\
                    .loc[:,['quarter','peer_set_ordered_revenue','brand_revenue_share_delta_index']]\
                    .iloc[-4:,:]\
                    .astype({'peer_set_ordered_revenue':'int64','brand_revenue_share_delta_index':'int'})\
                    .rename(
                        columns = {
                            'peer_set_ordered_revenue':'Peer Set Sales Revenue (axis-less)',
                            'brand_revenue_share_delta_index':'Sales Share Index'
                        }
                    )
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=2, 
    chart_name='sales_chart', 
    series_names = ['Peer Set Sales Revenue (axis-less)','Sales Share Index'], 
    df=chart_input, 
    x_column='quarter', 
    y_columns=['Peer Set Sales Revenue (axis-less)','Sales Share Index']
)

##### Chart monthly

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','peer_set_ordered_revenue','brand_revenue_share_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'peer_set_ordered_revenue':'int64',
                             'brand_revenue_share_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'peer_set_ordered_revenue':'Peer Set Sales Revenue (axis-less)',
                            'brand_revenue_share_delta_index':'Sales Share Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=1, 
    chart_name='sales_chart', 
    series_names = ['Peer Set Sales Revenue (axis-less)','Sales Share Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Peer Set Sales Revenue (axis-less)','Sales Share Index']
)

#### charts glance views

##### Chart quarterly

In [ ]:
chart_input = df_quarterly\
                    .loc[:,['quarter','peer_set_glance_views','brand_gv_share_delta_index']]\
                    .iloc[-4:,:]\
                    .astype({'peer_set_glance_views':'int64','brand_gv_share_delta_index':'int'})\
                    .rename(
                        columns = {
                            'peer_set_glance_views':'Peer Set Glance Views (axis-less)',
                            'brand_gv_share_delta_index':'Glance Views Share Index'
                        }
                    )
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=2, 
    chart_name='gvs_chart', 
    series_names = ['Peer Set Glance Views (axis-less)','Glance Views Share Index'], 
    df=chart_input, 
    x_column='quarter', 
    y_columns=['Peer Set Glance Views (axis-less)','Glance Views Share Index']
)

##### Chart monthly

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','peer_set_glance_views','brand_gv_share_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'peer_set_glance_views':'int64',
                             'brand_gv_share_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'peer_set_glance_views':'Peer Set Glance Views (axis-less)',
                            'brand_gv_share_delta_index':'Glance Views Share Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input


In [ ]:
pres.change_bar_chart(
    slide_number=1, 
    chart_name='gvs_chart', 
    series_names = ['Peer Set Glance Views (axis-less)','Glance Views Share Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Peer Set Glance Views (axis-less)','Glance Views Share Index']
)


#### charts display

##### Chart quarterly

In [ ]:
chart_input = df_quarterly\
                    .loc[:,['quarter','peer_set_total_display_impressions','brand_display_sov_delta_index']]\
                    .iloc[-4:,:]\
                    .astype({'peer_set_total_display_impressions':'int64','brand_display_sov_delta_index':'int'})\
                    .rename(
                        columns = {
                            'peer_set_total_display_impressions':'Peer Set Display Impressions (axis-less)',
                            'brand_display_sov_delta_index':'Display SOV Index'
                        }
                    )
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=2, 
    chart_name='display_chart', 
    series_names = ['Peer Set Display Impressions (axis-less)','Display SOV Index'], 
    df=chart_input, 
    x_column='quarter', 
    y_columns=['Peer Set Display Impressions (axis-less)','Display SOV Index']
)

##### Chart monthly

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','peer_set_total_display_impressions','brand_display_sov_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'peer_set_total_display_impressions':'int64',
                             'brand_display_sov_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'peer_set_total_display_impressions':'Peer Set Display Impressions (axis-less)',
                            'brand_display_sov_delta_index':'Display SOV Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input


In [ ]:
pres.change_bar_chart(
    slide_number=1, 
    chart_name='display_chart', 
    series_names = ['Peer Set Display Impressions (axis-less)','Display SOV Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Peer Set Display Impressions (axis-less)','Display SOV Index']
)


#### charts search

##### Chart quarterly

In [ ]:
chart_input = df_quarterly\
                    .loc[:,['quarter','peer_set_total_search_impressions','brand_search_sov_delta_index']]\
                    .iloc[-4:,:]\
                    .astype({'peer_set_total_search_impressions':'int64','brand_search_sov_delta_index':'int'})\
                    .rename(
                        columns = {
                            'peer_set_total_search_impressions':'Peer Set Search Impressions (axis-less)',
                            'brand_search_sov_delta_index':'Search SOV Index'
                        }
                    )
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=2, 
    chart_name='search_chart', 
    series_names = ['Peer Set Search Impressions (axis-less)','Search SOV Index'], 
    df=chart_input, 
    x_column='quarter', 
    y_columns=['Peer Set Search Impressions (axis-less)','Search SOV Index']
)

##### Chart monthly

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','peer_set_total_search_impressions','brand_search_sov_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'peer_set_total_search_impressions':'int64',
                             'brand_search_sov_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'peer_set_total_search_impressions':'Peer Set Search Impressions (axis-less)',
                            'brand_search_sov_delta_index':'Search SOV Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input


In [ ]:
pres.change_bar_chart(
    slide_number=1, 
    chart_name='search_chart', 
    series_names = ['Peer Set Search Impressions (axis-less)','Search SOV Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Peer Set Search Impressions (axis-less)','Search SOV Index']
)


#### charts overlap 1

##### Chart monthly: sales vs glance views

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','brand_revenue_share_delta_index','brand_gv_share_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'brand_revenue_share_delta_index':'int64',
                             'brand_gv_share_delta_index':'int'
                            })\
                    .rename(
                        columns = {
                            'brand_revenue_share_delta_index':'Sales Share Index',
                            'brand_gv_share_delta_index':'Glance Views Share Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=1, 
    chart_name='overlap_chart_1', 
    series_names = ['Sales Share Index','Glance Views Share Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Sales Share Index','Glance Views Share Index']
)


#### charts overlap 2

##### Chart monthly: glance views vs search/display

In [ ]:
chart_input = df_monthly\
                    .loc[:,['month_starting_date','brand_gv_share_delta_index','brand_display_sov_delta_index','brand_search_sov_delta_index']]\
                    .iloc[-12:,:]\
                    .astype({'month_starting_date':'datetime64[ns]',
                             'brand_gv_share_delta_index':'int',
                             'brand_display_sov_delta_index':'int',
                             'brand_search_sov_delta_index':'int'                             
                            })\
                    .rename(
                        columns = {
                            'brand_gv_share_delta_index':'Glance Views Share Index',
                            'brand_display_sov_delta_index':'Display SOV Index',
                            'brand_search_sov_delta_index':'Search SOV Index'
                        }
                    )
chart_input['month_starting_date'] = chart_input.month_starting_date.dt.strftime('%b')
chart_input

In [ ]:
pres.change_bar_chart(
    slide_number=1, 
    chart_name='overlap_chart_2', 
    series_names = ['Glance Views Share Index','Display SOV Index','Search SOV Index'], 
    df= chart_input, 
    x_column='month_starting_date', 
    y_columns=['Glance Views Share Index','Display SOV Index','Search SOV Index']
)


#### Export

In [ ]:
## export results
export_path = "./results/" + input_dic['local'] + ' ' + input_dic['brand_name'] + '/'  + input_dic['category_friendly_name'] + '/output'
export_file = input_dic['local']  + '_' + input_dic['brand_name'] + '_brand_performance_report_v2.pptx'
pres.save(export_path + '/' + export_file)
del pres